# Recent Images Microservice

This microservice can be used to request multiple tile urls, thumbnail urls and metadata from Google Earth Engine (GEE) in a fast, asynchronus way.

### Overview

On the GFW map page users can currently use the *Sentinel service* to inspect a single satellite image tile for a given area of interest when at high zoom level. The tile url of the image with the lowest ```cloud_score``` within a selected date range is requested from GEE and used on the front end to display the image.

This microservice builds on this idea by requesting all images in the date range asynchronously as well as the corresponding thumbnail urls and associated metadata.

***

### Intended Useage

The microservice is intented to be implemented in 3 stages on the front end:

1) First, do a GET request for the metadata from a GEE image collection (ordered by CLOUDY_PIXEL_PERCENTAGE), as well as the tile url for the best image (i.e. the one with the lowest CLOUDY_PIXEL_PERCENTAGE score)

2) Next POST a payload containing the ```'source'``` values of all images to the ```/v1/recent-tiles/tiles``` endpoint, which returns the tile urls for the associated images.

3) Finally POST the same payload to the ```/v1/recent-tiles/thumbs``` endpoint, which returns all associated thumbnail urls.

***


### Endpoints

The microservice has 3 endpoints:

**/v1/recent-tiles** (GET)
- *Returns tile metadata and a single tile url*
- url params: ```lat, lon, start, end```

**/v1/recent-tiles/tiles** (POST)
- *Returns all tile urls*
- payload: ```list of JSON objects containing 'source' value```

**/v1/recent-tiles/thumbs** (POST)
- *Returns all thumb urls*
- payload: ```list of JSON objects containing 'source' value```





In [9]:
import folium
import os
import ee
import json
import requests
import math
from scipy import misc
import shutil
from pprint import pprint
import requests
from IPython.display import Image

# Request 1

First query requests data for the following params, for example:

```json
    url = f"https://production-api.globalforestwatch.org/v1/recent-tiles"
    params = {'lat':'-16.644','lon':'28.266', 'start':'2016-01-01', 'end': "2016-01-08"}
```

And returns an object of the following format:

```json
"data": [
    {
        "attributes": {
        "boundary_tiles": "https://earthengine.googleapis.com/map/4b1b...",
        "cloud_score": 13.1214,
        "date_time": "2017-01-31 11:52:11Z",
        "instrument": "Sentinel-2A",
        "source": "COPERNICUS/S2/20170131T115211_20170131T115306_T28RCS",
        "thumbnail_url": Null,
        "tile_url": Null
        },
        "id": Null,
        "type": "recent_tiles_url"
    }

```

There may be multiple values in the ```'data'``` list (expect one image per 8 days approx.)


In [10]:
%%time
# Request metadata (6 month period)

url = f"https://production-api.globalforestwatch.org/v1/recent-tiles"
params= {'lat':'-16.644','lon':'28.266', 'start':'2016-01-01', 'end': "2016-9-01"}
r = requests.get(url, params=params)
print(r.url)
r.status_code

data = r.json().get('data')

print(f"Returned {len(data)} items. \nPreview of first 3...")
pprint(data[0:3])

https://production-api.globalforestwatch.org/v1/recent-tiles?lat=-16.644&lon=28.266&start=2016-01-01&end=2016-9-01
Returned 21 items. 
Preview of first 3...
[{'attributes': {'boundary_url': 'https://earthengine.googleapis.com/map/4b1b9c6f82d50796562521502bc4d9a2/{z}/{x}/{y}?token=3b62d76749d0c096f8196be749040215',
                 'cloud_score': 0.4339,
                 'date_time': '2016-03-17 11:53:14Z',
                 'instrument': 'Sentinel-2A',
                 'source': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS',
                 'thumbnail_url': None,
                 'tile_url': 'https://earthengine.googleapis.com/map/daf140b2b3e97b1be86bb49893314cd0/{z}/{x}/{y}?token=8323e8319395285243f95913c69d9383'},
  'id': None,
  'type': 'recent_tiles_data'},
 {'attributes': {'boundary_url': 'https://earthengine.googleapis.com/map/4b1b9c6f82d50796562521502bc4d9a2/{z}/{x}/{y}?token=3b62d76749d0c096f8196be749040215',
                 'cloud_score': 3.1135,
                 'da

In [11]:
# In the returned data, one of the rows of data will contain a valid tile url, which
# should be used to immediatley display an image...
for row in data:
    if row.get('attributes').get('tile_url') is not None:
        tile_url = row.get('attributes').get('tile_url')
        print(tile_url)

https://earthengine.googleapis.com/map/daf140b2b3e97b1be86bb49893314cd0/{z}/{x}/{y}?token=8323e8319395285243f95913c69d9383


In [21]:
quick_map = folium.Map(location=[float(params['lon']), float(params['lat'])], zoom_start=9, tiles='Mapbox Bright' )
quick_map.add_tile_layer(tiles=tile_url, max_zoom=19, min_zoom=6, attr="Live EE tiles")
#quick_map

![](./pics/ten1.png)

# Creating Payload for POST

On the front end, the response must be used to buld a list of objects containing the ```'source'``` values of the images in the collection.

*e.g.*

```json
{'source_data':
    
    [
        {"source": "COPERNICUS/S2/20170131T11..."},
        {"source": "COPERNICUS/S2/20170131T12..."},
        {"source": "COPERNICUS/S2/20170131T13..."},    
        
        ...
      
        {"source": "COPERNICUS/S2/20170131T1N..."}
    ]
 }

```

In [14]:
#example of how to construct the source list

source_list = [{'source': d.get('attributes').get('source')} for d in data]    
    
source_list

[{'source': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS'},
 {'source': 'COPERNICUS/S2/20160605T115314_20160605T200612_T28RCS'},
 {'source': 'COPERNICUS/S2/20160416T115220_20160416T172059_T28RCS'},
 {'source': 'COPERNICUS/S2/20160725T115222_20160725T184330_T28RCS'},
 {'source': 'COPERNICUS/S2/20160824T115222_20160824T200405_T28RCS'},
 {'source': 'COPERNICUS/S2/20160715T115224_20160715T200356_T28RCS'},
 {'source': 'COPERNICUS/S2/20160506T115225_20160506T184707_T28RCS'},
 {'source': 'COPERNICUS/S2/20160814T115222_20160814T184201_T28RCS'},
 {'source': 'COPERNICUS/S2/20160327T115315_20160327T172523_T28RCS'},
 {'source': 'COPERNICUS/S2/20160615T115223_20160615T183608_T28RCS'},
 {'source': 'COPERNICUS/S2/20160804T115238_20160804T200426_T28RCS'},
 {'source': 'COPERNICUS/S2/20160117T115842_20160117T172446_T28RCS'},
 {'source': 'COPERNICUS/S2/20160526T115225_20160526T184022_T28RCS'},
 {'source': 'COPERNICUS/S2/20160406T115217_20160406T185930_T28RCS'},
 {'source': 'COPERNICUS/S2/2016062

### Request tiles using the payload

*IMPORTANT*

- The json must have the key ```'source_data'```

- It must have the following header: ```headers={'Content-Type': 'application/json'}```

In [15]:
%%time

url = f"https://production-api.globalforestwatch.org/v1/recent-tiles/tiles"
payload = {'source_data': source_list}
r = requests.post(url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
r.status_code

tile_data = r.json().get('data').get('attributes')
print(f"Returned tile urls for {len(tile_data)} ids.\nPreviewing first 3 items...")
pprint(tile_data[0:3])

Returned tile urls for 21 ids.
Previewing first 3 items...
[{'source_id': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS',
  'tile_url': 'https://earthengine.googleapis.com/map/daf140b2b3e97b1be86bb49893314cd0/{z}/{x}/{y}?token=772bdcd0abf69c06ccaaa27ed63f4bb5'},
 {'source_id': 'COPERNICUS/S2/20160605T115314_20160605T200612_T28RCS',
  'tile_url': 'https://earthengine.googleapis.com/map/4a5c95e26f3561323033c9e78aeea33c/{z}/{x}/{y}?token=e9448686d8e6919c4880cfe73f3a9842'},
 {'source_id': 'COPERNICUS/S2/20160416T115220_20160416T172059_T28RCS',
  'tile_url': 'https://earthengine.googleapis.com/map/ab0b08438d564fc4822fbb3fd50c7f9a/{z}/{x}/{y}?token=ddeef908a8c5bd5834315951f50ab8cd'}]
CPU times: user 22.4 ms, sys: 3.18 ms, total: 25.6 ms
Wall time: 1.2 s


### Tiles Response

At this stage the data now looks like:

```json
{'data': {
    'attributes': [{"source": "COPERNICUS/S2/20170131T11...",
                    "tile_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },{
                    "source": "COPERNICUS/S2/20170131T12...",
                    "tiles_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },           
                      ...
      
                    {
                    "source": "COPERNICUS/S2/20170131T1N...",
                    "tiles_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   }],
          'id': None,
          'type': 'recent_tiles_url'}}
          }
}
```

### Thumbnail service

The thumbnail images will be needed. We must request them seperatley.

In [16]:
%%time

# Request thumbs using the payload

url = f"https://production-api.globalforestwatch.org/v1/recent-tiles/thumbs"
payload = {'source_data':source_list}
r = requests.post(url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
r.status_code

thumb_data = r.json().get('data').get('attributes')

pprint(thumb_data[0:3])

[{'source': 'COPERNICUS/S2/20160317T115314_20160317T172609_T28RCS',
  'thumbnail_url': 'https://earthengine.googleapis.com/api/thumb?thumbid=f8d1d5c66a6d3e7dc30002e663242830&token=778f906027b341e1807751fb9cd7d241'},
 {'source': 'COPERNICUS/S2/20160605T115314_20160605T200612_T28RCS',
  'thumbnail_url': 'https://earthengine.googleapis.com/api/thumb?thumbid=fbcea1cc967a46d14c48c2367c0bf5a3&token=b093d5237097bb345384fe22a451435d'},
 {'source': 'COPERNICUS/S2/20160416T115220_20160416T172059_T28RCS',
  'thumbnail_url': 'https://earthengine.googleapis.com/api/thumb?thumbid=1bfbf427df20fd31c68ab9f575ff04f2&token=b8aef3b042394d922f71caf6027fd162'}]
CPU times: user 20.2 ms, sys: 3.79 ms, total: 24 ms
Wall time: 1.66 s


In [17]:
# E.g. of a thumbnail

for row in thumb_data:
    thumb_url = row.get('thumbnail_url')
    break
    
Image(url=thumb_url)      

### Build a JSON object to work from

At this stage you can construct a json object like the following:

```json
{'data': {
    'attributes': [{"source": "COPERNICUS/S2/20170131T11...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },{
                    "source": "COPERNICUS/S2/20170131T12...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   },           
                      ...
      
                    {
                    "source": "COPERNICUS/S2/20170131T1N...",
                    "thumbnail_url": "https://earthengine.googleapis.com/api/thumb?thumbid=..."
                   }],
          'id': None,
          'type': 'recent_thumbs_url'}}
          }
}
```

# Image Example for Map

In [18]:
# Returned First Image and Boundary

dt = data[0].get('attributes').get('date_time')
boundary = data[0].get('attributes').get('boundary_url')
sentinel_image = data[0].get('attributes').get('tile_url')

In [22]:
sentinel_map = folium.Map(location=[float(params['lon']), float(params['lat'])], zoom_start=9, tiles='Mapbox Bright' )
sentinel_map.add_tile_layer(tiles=sentinel_image, max_zoom=19, min_zoom=6, attr="Live EE tiles")
sentinel_map.add_tile_layer(tiles=boundary, max_zoom=19, min_zoom=6, attr="Live EE tiles")
#sentinel_map

![](./pics/ten2.png)